# Pholly

Welcome to the Pholly interactive notebook - this is an introduction to using resilience patterns with F# and Pholly which is itself based on Polly.

## What is Pholly

Pholly provides a more natural way to use Polly from within F# and uses a number of F# language features to provide an expressive DSL type approach to using common resilience patterns. Currently the three Polly reactive resilience patterns are supported:

|Pattern|Purpose|
|-------|-------|
|Retry|Many faults are transient and may self-correct after a short delay and the retry pattern will attempt to retry the operation until it succeeds or a failure condition is met|
|Circuit Breaker|When a system is seriously struggling, failing fast is better than making users/callers wait. The circuit breaker can protect a faulting system from overloading and help it recover.|
|Fallback|Things can and do fail and it can be useful to provide a fallback|

## Pholly Principles

Pholly uses the F# Result<> type for its error control flow rather than the exceptions you normally see in Polly examples (which are often based on C#).

## Retry Policy

The retry policy will retry a function until either it succeeds or a constraint on the retry policy is met. The code below shows a simple retry operation that will attempt to roll a dice until a 5 or higher is rolled. It will keep retrying forever.

In [1]:
#!fsharp
#r "nuget: Pholly"

open Pholly
open Retry
open System

let log msg = printf "%s" msg

let random = Random((DateTime.UtcNow.Ticks % (Int32.MaxValue |> int64)) |> int32)
let rollDice () = random.Next(1,7)
let isSuccessfulDiceRoll diceRoll = if diceRoll >= 5 then diceRoll |> Ok else "Out of range" |> Error

let retryForever = Policy.retry [ retry forever ]
match (rollDice >> isSuccessfulDiceRoll) |> retryForever with
| Ok value -> sprintf "Success - returned %d" value |> log
| Error error -> sprintf "ERROR: %s" error |> log

Its not usually helpful to retry an operation forever and so the code below builds on our forever example shows a simple retry policy that will roll a dice and continue to retry until a 5 or higher is rolled or 10 errors occur. Before each retry it will print a message with the retry count.

In [1]:
#!fsharp
#r "nuget: Pholly"

open Pholly
open Retry
open System

let log msg = printf "%s" msg
let random = Random((DateTime.UtcNow.Ticks % (Int32.MaxValue |> int64)) |> int32)
let rollDice () = random.Next(1,7)
let isSuccessfulDiceRoll diceRoll = if diceRoll >= 5 then diceRoll |> Ok else "Out of range" |> Error

let retry = Policy.retry [
    retry (upto 10<times>)
    beforeEachRetry (fun _ retryAttempt _ -> sprintf "Retrying attempt %d" retryAttempt |> log)
]

match (rollDice >> isSuccessfulDiceRoll) |> retry with
| Ok value -> sprintf "Success - returned %d" value |> log
| Error error -> sprintf "ERROR: %s" error |> log


The next example shows an asynchronous retry policy for an asyncrhonous workload that also includes a back off sequence - each retry attempt will result in the next delay in the series. If their are more retries than their are specified intervals then the out of range retries will use the final interval.

In [1]:
#!fsharp
#r "nuget: Pholly"

open Pholly
open Retry
open System

let log msg = printf "%s" msg
let random = Random((DateTime.UtcNow.Ticks % (Int32.MaxValue |> int64)) |> int32)
let rollDice () = random.Next(1,7)
let isSuccessfulDiceRoll diceRoll = if diceRoll >= 5 then diceRoll |> Ok else "Out of range" |> Error

let retry = Policy.retryAsync [
    retry (upto 10<times>)
    withIntervalsOf [50<ms> ; 500<ms> ; 1000<ms>]
    beforeEachRetry (fun _ retryAttempt _ -> sprintf "Retrying attempt %d" retryAttempt |> log)
]

match (async { return (rollDice () |> isSuccessfulDiceRoll) }) |> retry |> Async.RunSynchronously with
| Ok value -> sprintf "Success - returned %d" value |> log
| Error error -> sprintf "ERROR: %s" error |> log


## Circuit Breaker

The circuit breaker policy is a good way to keep a failing system space to recover - for example their's no point pounding on an API endpoint that is already overloaded and returning errors as that will likely hinder its recovery.

The example below creates a circuit breaker that will "trip" into its open state after 3 consecutive errors meaning the fourth call to our protected function will not be made:

In [1]:
#!fsharp
#r "nuget: Pholly"

open Pholly
open CircuitBreaker
open System

let log msg = printf "%s" msg
let outputBreakerResult result = match result with | Ok s -> log s | Error e -> log (sprintf "Circuit breaker error: %s\n" e)

let execute,_,_ =
    Policy.circuitBreaker [
        breakOn 3<consecutiveErrors>
        circuitOpenResult ("Circuit is open" |> Error)
    ]

execute (fun _ -> "Success" |> Ok) |> outputBreakerResult
execute (fun _ -> "Consecutive error 1" |> Error) |> outputBreakerResult
execute (fun _ -> "Consecutive error 2" |> Error) |> outputBreakerResult
execute (fun _ -> "Consecutive error 3 - should trip breaker" |> Error) |> outputBreakerResult
execute (fun _ -> "Consecutive error 4 - should not be called" |> Error) |> outputBreakerResult


Success

Circuit breaker error: Consecutive error 1


Circuit breaker error: Consecutive error 2


Circuit breaker error: Consecutive error 3 - should trip breaker


Circuit breaker error: Circuit is open


The next example breaks up our 3 consecutive errors with another successful result. This means the circuit breaker never trips:

In [1]:
#!fsharp
#r "nuget: Pholly"

open Pholly
open CircuitBreaker
open System

let log msg = printf "%s" msg
let outputBreakerResult result = match result with | Ok s -> log s | Error e -> log (sprintf "Circuit breaker error: %s\n" e)

let execute,_,_ =
    Policy.circuitBreaker [
        breakOn 3<consecutiveErrors>
        circuitOpenResult ("Circuit is open" |> Error)
    ]

execute (fun _ -> "Success" |> Ok) |> outputBreakerResult
execute (fun _ -> "Consecutive error a.1" |> Error) |> outputBreakerResult
execute (fun _ -> "Consecutive error a.2" |> Error) |> outputBreakerResult
execute (fun _ -> "Success" |> Ok) |> outputBreakerResult
execute (fun _ -> "Consecutive error b.1" |> Error) |> outputBreakerResult
execute (fun _ -> "Consecutive error b.2" |> Error) |> outputBreakerResult

## Thanks

Many thanks to the creators of [Polly](https://github.com/App-vNext/Polly) itself - I've used it in many production systems and I've used some of their descriptions of the patterns here.